In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import os

In [2]:
os.chdir('/content/drive/MyDrive/Deep Learning/Q1')

In [ ]:
!ls -lha kaggle.json
!pip install -q kaggle # installing the kaggle package
!mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
!cp kaggle.json ~/.kaggle/ # move the key to the folder
!pwd # checking the present working directory

-rw------- 1 root root 69 May 20 14:09 kaggle.json
/content/drive/MyDrive/Deep Learning/Q1


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d misrakahmed/vegetable-image-dataset

 99% 527M/534M [00:04<00:00, 181MB/s]
100% 534M/534M [00:04<00:00, 131MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Deep Learning/Q1/vegetable-image-dataset.zip','r')
zip_ref.extractall(os.curdir)
zip_ref.close() 

In [ ]:
#Installing Albmentation for data transformation

In [ ]:
# !pip install -U albumentations

In [ ]:
import os 
import albumentations as A
import cv2 

In [3]:
train_file_path = os.path.join(os.curdir,'Vegetable Images','train')

In [4]:
vegetable_name = os.listdir(train_file_path)

In [ ]:
#transformation Object 
transform = A.Compose([
    A.RandomCrop(width=100, height=120),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.VerticalFlip(p=0.5),
]) 

train_aug = os.path.join(os.curdir,'train_augmentation')
if not os.path.exists(train_aug): #Creating Train Augmentation Folder
    os.makedirs(train_aug)

for veg in vegetable_name:
    veg_name_dir_path = os.path.join(train_file_path,veg) #Original Vegetable Folder path
    veg_name_aug_dir_path = os.path.join(train_aug,veg) #Augmented Veggetable path
    
    if not os.path.exists(veg_name_aug_dir_path):
      os.makedirs(veg_name_aug_dir_path) #Creating Vegetable Folder path

    count = 1
    for img in os.listdir(veg_name_dir_path): #Taking all Imeges
        image = cv2.imread(os.path.join(veg_name_dir_path,img))
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        augmented_image = transform(image=image)['image'] #tansforming our image
        cv2.imwrite(os.path.join(veg_name_aug_dir_path,f'{count}.jpg'),augmented_image) #saving our image
        count+= 1
  






In [5]:
train_path = os.path.join(os.curdir,'train_augmentation') #Train Folder 
val_path = os.path.join(os.curdir,'Vegetable Images','validation') # Validation Folder 
train_aug = os.path.join(os.curdir,'train_augmentation')

In [ ]:
#Now after augmenting the data have to convert into numpy array 
#Then Model bulding 
#Then model Training

In [6]:
#importing libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchsummary import summary
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [7]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


device(type='cuda')

In [8]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((152,152)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [9]:
train_path = os.path.join(os.curdir,'train_augmentation') #Train Folder 
val_path = os.path.join(os.curdir,'Vegetable Images','validation') # Validation Folder 

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
val_loader=DataLoader(
    torchvision.datasets.ImageFolder(val_path,transform=transformer),
    batch_size=32, shuffle=False
)

In [10]:
#Model Building

In [11]:
class Cnn(nn.Module):
    def __init__(self,):
      super(Cnn,self).__init__()

      #input shape --> (64,3,152,152)
      self.conv1 = nn.Conv2d(in_channels=3,out_channels=8,kernel_size=3) #150,150
      self.relu_1 = nn.ReLU()
      self.conv2 = nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3) #148,148
      self.relu_2 = nn.ReLU()

      self.maxpoll_1 = nn.MaxPool2d(kernel_size=2) #74,74

      self.conv3 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3) #72,72
      self.relu_3 = nn.ReLU()
      self.conv4 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3) #70,70
      self.relu_4 = nn.ReLU()

      self.maxpoll_2 = nn.MaxPool2d(kernel_size=2) #35,35

      self.conv5 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3) #33,33
      self.relu_5 = nn.ReLU()

      self.fc = nn.Linear(in_features=33*33*128,out_features=len(vegetable_name))



    #Feed Forward 
    def forward(self,x):
      x = self.conv1(x)
      x = self.relu_1(x)
      x = self.conv2(x)
      x = self.relu_2(x)

      x = self.maxpoll_1(x)

      x = self.conv3(x)
      x = self.relu_3(x)
      x = self.conv4(x)
      x = self.relu_4(x)

      x = self.maxpoll_2(x)

      x = self.conv5(x)
      x = self.relu_5(x)
      #FLatten
      x = x.view(-1,33*33*128)

      x = self.fc(x)

      return x 


In [12]:
model=Cnn().to(device) #insilizing the model 


In [13]:
summary(model,(3,152,152)) #geting model summery 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 150, 150]             224
              ReLU-2          [-1, 8, 150, 150]               0
            Conv2d-3         [-1, 16, 148, 148]           1,168
              ReLU-4         [-1, 16, 148, 148]               0
         MaxPool2d-5           [-1, 16, 74, 74]               0
            Conv2d-6           [-1, 32, 72, 72]           4,640
              ReLU-7           [-1, 32, 72, 72]               0
            Conv2d-8           [-1, 64, 70, 70]          18,496
              ReLU-9           [-1, 64, 70, 70]               0
        MaxPool2d-10           [-1, 64, 35, 35]               0
           Conv2d-11          [-1, 128, 33, 33]          73,856
             ReLU-12          [-1, 128, 33, 33]               0
           Linear-13                   [-1, 16]       2,230,288
Total params: 2,328,672
Trainable param

In [14]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001) #inislizing the optimizer
loss_function=nn.CrossEntropyLoss() # insilizing the loss_function 

In [15]:
num_epochs=50 #number of epocs to train 

In [16]:
train_count=len(glob.glob(train_path+'/**/*.jpg')) #total images in train folder
val_count=len(glob.glob(val_path+'/**/*.jpg')) #total images in val folder 

print(train_count,val_count)

15000 2986


In [ ]:
# Start of training 

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    val_accuracy=0.0
    for i, (images,labels) in enumerate(val_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        val_accuracy+=int(torch.sum(prediction==labels.data))
    
    val_accuracy=val_accuracy/val_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' VAl Accuracy: '+str(val_accuracy))
    
    #Save the best model
    if val_accuracy>best_accuracy:
        torch.save(model,os.path.join(os.curdir,'model.pt'))
        best_accuracy=val_accuracy

In [10]:
from PIL import Image

In [11]:
image = Image.open('/content/drive/MyDrive/Deep Learning/Q1/Vegetable Images/test/Bean/0001.jpg')

In [ ]:
image_tensor = transformer(image).float()
image_tensor=image_tensor.unsqueeze_(0)
image_tensor = image_tensor.cuda()

In [ ]:
input = Variable(image_tensor)

In [ ]:
model = torch.load('/content/drive/MyDrive/Deep Learning/Q1/model.pt')

In [ ]:
model.eval()

Cnn(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1))
  (relu_1): ReLU()
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (relu_2): ReLU()
  (maxpoll_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu_3): ReLU()
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (relu_4): ReLU()
  (maxpoll_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (relu_5): ReLU()
  (fc): Linear(in_features=139392, out_features=15, bias=True)
)

In [ ]:
output = model(input)

In [ ]:
index = output.cpu().data.numpy().argmax()

In [ ]:
vegetable_name[index]

'Bitter_Gourd'

In [ ]:
#Converting model to onnx format

In [ ]:
#!pip install torch torchvision onnx onnxruntime


In [ ]:
import torch.onnx as onnx

In [ ]:
image_input = torch.randn(1,3,152,152)
image_input = image_input.cuda()
onnx.export(model,image_input,os.path.join(os.curdir,'ONNX Model','model.onnx'))

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
import onnx

In [ ]:
onnx_model = onnx.load('/content/drive/MyDrive/Deep Learning/Q1/ONNX Model/model.onnx')

# Check the model for correctness
onnx.checker.check_model(onnx_model)

In [5]:

#!pip install onnxruntime




In [6]:
import onnxruntime

In [7]:
model_path = '/content/drive/MyDrive/Deep Learning/Q1/ONNX Model/model.onnx' #loding the model path 

In [9]:
session = onnxruntime.InferenceSession(model_path) # loading the model 

In [12]:
#preprocessing the the data 

In [13]:
from PIL import Image

In [36]:
image = Image.open("/content/drive/MyDrive/Deep Learning/Q1/Vegetable Images/test/Cabbage/0929.jpg") #loading the image 

In [37]:
type(image)

PIL.JpegImagePlugin.JpegImageFile

In [50]:
#transforming the image and converting it into a tensor 
transformer=transforms.Compose([
    transforms.Resize((152,152)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [51]:
image_tensor = transformer(image)

In [40]:
image_tensor.shape

torch.Size([3, 152, 152])

In [41]:
image_tensor = image_tensor.unsqueeze_(0) #adding another dimmension 

In [42]:
image_tensor.shape

torch.Size([1, 3, 152, 152])

In [43]:
import numpy as np 
image_tensor = np.array(image_tensor) #converting it into a numpy array

In [44]:
image_tensor.shape

(1, 3, 152, 152)

In [45]:
output = session.run(None,{'input.1':image_tensor})

In [28]:
session.get_inputs()[0].name

'input.1'

In [58]:
output

array([  1.8014171,   3.5160897,  -1.4940233,   0.9165438,   2.9631984,
         2.8010273,  -3.949736 , -15.1769495,   1.6306075,   0.901724 ,
        -3.429121 , -15.26182  ,  -1.653359 ,  -1.1608636,   1.0911462],
      dtype=float32)

In [59]:
np.argmax(output[0][0])

1